In [ ]:
import os
import shutil
import random
import zipfile
import tarfile
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings

# Configurações de exibição e filtros de avisos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Silencia logs informativos do TF
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*metadata.*") # Silencia avisos de metadados de imagens

print("Ambiente configurado com sucesso.")

In [ ]:
#Nota: Em MLOps, esta etapa seria automatizada por um script de data ingestion.

# Cria a estrutura de diretórios para os modelos pré-treinados
import os

model_dirs = [
    '/tmp/model-balanced/variables',
    '/tmp/history-balanced',
    '/tmp/model-imbalanced/variables',
    '/tmp/history-imbalanced',
    '/tmp/model-augmented/variables',
    '/tmp/history-augmented'
]

for dir_path in model_dirs:
    os.makedirs(dir_path, exist_ok=True)

print("Estrutura de diretórios criada. Iniciando downloads...\n")

# Definição dos links e destinos
# Datasets originais
#!wget https://storage.googleapis.com/mlep-public/course_1/week2/kagglecatsanddogs_3367a.zip
#!wget https://storage.googleapis.com/mlep-public/course_1/week2/CUB_200_2011.tar

# Modelos pré-treinados
# O parâmetro -P define o diretório de destino

!wget -q -P /tmp/model-balanced/ https://storage.googleapis.com/mlep-public/course_1/week2/model-balanced/saved_model.pb
!wget -q -P /tmp/model-balanced/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-balanced/variables/variables.data-00000-of-00001
!wget -q -P /tmp/model-balanced/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-balanced/variables/variables.index
!wget -q -P /tmp/history-balanced/ https://storage.googleapis.com/mlep-public/course_1/week2/history-balanced/history-balanced.csv

!wget -q -P /tmp/model-imbalanced/ https://storage.googleapis.com/mlep-public/course_1/week2/model-imbalanced/saved_model.pb
!wget -q -P /tmp/model-imbalanced/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-imbalanced/variables/variables.data-00000-of-00001
!wget -q -P /tmp/model-imbalanced/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-imbalanced/variables/variables.index
!wget -q -P /tmp/history-imbalanced/ https://storage.googleapis.com/mlep-public/course_1/week2/history-imbalanced/history-imbalanced.csv

!wget -q -P /tmp/model-augmented/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/saved_model.pb
!wget -q -P /tmp/model-augmented/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.data-00000-of-00001
!wget -q -P /tmp/model-augmented/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.index
!wget -q -P /tmp/history-augmented/ https://storage.googleapis.com/mlep-public/course_1/week2/history-augmented/history-augmented.csv

print("\n✅ Downloads concluídos!")

In [ ]:
# Definição de caminhos
cats_and_dogs_zip = 'kagglecatsanddogs_3367a.zip'
caltech_birds_tar = 'CUB_200_2011.tar'

base_dir = '/tmp/data'

In [ ]:
# Extração dos arquivos compactados
with zipfile.ZipFile(cats_and_dogs_zip, 'r') as my_zip:
  my_zip.extractall(base_dir)
with tarfile.open(caltech_birds_tar, 'r') as my_tar:
  my_tar.extractall(base_dir)

In [ ]:
#Mapeamento dos diretórios de Cães e Gatos (já vêm organizados)
base_dogs_dir = os.path.join(base_dir, 'PetImages/Dog')
base_cats_dir = os.path.join(base_dir, 'PetImages/Cat')

print(f"Existem {len(os.listdir(base_dogs_dir))} imagens de cães")
print(f"Existem {len(os.listdir(base_cats_dir))} imagens de gatos")

In [ ]:
raw_birds_dir = '/tmp/data/CUB_200_2011/images'
base_birds_dir = os.path.join(base_dir, 'PetImages/Bird')

os.makedirs(base_birds_dir, exist_ok=True)

moved_count = 0
skipped_count = 0

for subdir in os.listdir(raw_birds_dir):
    subdir_path = os.path.join(raw_birds_dir, subdir)

    if not os.path.isdir(subdir_path):
        continue

    for image in os.listdir(subdir_path):
        src = os.path.join(subdir_path, image)

        # Novo nome = nome da espécie + nome original da imagem
        new_name = f"{subdir}__{image}"
        dst = os.path.join(base_birds_dir, new_name)

        if not os.path.exists(dst):
            shutil.move(src, dst)
            moved_count += 1
        else:
            skipped_count += 1

print(f"Pássaros movidos com sucesso: {moved_count}")
print(f"Pássaros ignorados (nome duplicado): {skipped_count}")
print(f"Total de imagens em Bird/: {len(os.listdir(base_birds_dir))}")

In [ ]:
from IPython.display import Image, display

# Exibe uma amostra de cada classe para validação visual
# Isso é importante para detectar problemas como imagens corrompidas ou mal rotuladas

print("Exemplo de uma imagem de gato:")
display(Image(filename=f"{os.path.join(base_cats_dir, os.listdir(base_cats_dir)[0])}"))
print("\nExemplo de uma imagem de cão:")
display(Image(filename=f"{os.path.join(base_dogs_dir, os.listdir(base_dogs_dir)[0])}"))
print("\nExemplo de uma imagem de passaro:")
display(Image(filename=f"{os.path.join(base_birds_dir, os.listdir(base_birds_dir)[0])}"))

In [ ]:
# Estrutura de diretórios para organização do dataset
# Padrão MLOps: /train/{classe} e /eval/{classe}
# Isso facilita o uso de geradores de dados do TensorFlow/Keras

train_eval_dirs = ['train/cats', 'train/dogs', 'train/birds', 
                   'eval/cats', 'eval/dogs', 'eval/birds']

# Cria os diretórios de forma segura (evita erro se já existirem)
for dir in train_eval_dirs:
    if not os.path.exists(os.path.join(base_dir, dir)):
        os.makedirs(os.path.join(base_dir, dir))

print("Estrutura de diretórios criada com sucesso.")

In [ ]:
# define a função para mover uma parte das imagens do diretorio original para os diretorios de treino e avaliação
def move_to_destination(origin, destination, percentage_split):
    """
    Move uma porcentagem das imagens de um diretório de origem para um destino.
    
    Parâmetros:
    -----------
    origin : str
        Caminho do diretório de origem contendo as imagens
    destination : str
        Caminho do diretório de destino
    percentage_split : float
        Porcentagem de imagens a mover (ex: 0.7 = 70%)
    
    Comportamento:
    --------------
    - Calcula quantas imagens mover baseado no percentual
    - Ordena os arquivos para garantir reprodutibilidade
    - Move (não copia) as imagens, economizando espaço em disco
    
    Exemplo:
    --------
    Se origin tem 1000 imagens e percentage_split=0.7:
    - Move as primeiras 700 imagens (ordenadas alfabeticamente)
    - Deixa 300 no diretório original
    """
    num_images = int(len(os.listdir(origin))*percentage_split)
    # zip() combina a lista de nomes com um contador
    # sorted() garante ordem consistente (importante para reprodutibilidade)
    for image_name, image_number in zip(sorted(os.listdir(origin)), range(num_images)):
        shutil.move(os.path.join(origin, image_name), destination)

In [ ]:
def move_to_destination(origin, destination, percentage_split):
    """
    Move uma porcentagem das imagens de um diretório de origem para um destino.
    
    Parâmetros:
    -----------
    origin : str
        Caminho do diretório de origem contendo as imagens
    destination : str
        Caminho do diretório de destino
    percentage_split : float
        Porcentagem de imagens a mover (ex: 0.7 = 70%)
    
    Comportamento:
    --------------
    - Calcula quantas imagens mover baseado no percentual
    - Ordena os arquivos para garantir reprodutibilidade
    - Move (não copia) as imagens, economizando espaço em disco
    - IDEMPOTENTE: ignora arquivos que já existem no destino
    
    Exemplo:
    --------
    Se origin tem 1000 imagens e percentage_split=0.7:
    - Move as primeiras 700 imagens (ordenadas alfabeticamente)
    - Deixa 300 no diretório original
    """
    # Lista apenas arquivos (ignora subdiretórios)
    files = [f for f in os.listdir(origin) if os.path.isfile(os.path.join(origin, f))]
    
    num_images = int(len(files) * percentage_split)
    
    moved = 0
    skipped = 0
    
    for image_name in sorted(files)[:num_images]:
        src = os.path.join(origin, image_name)
        dst = os.path.join(destination, image_name)
        
        # Verifica se o arquivo já existe no destino
        if not os.path.exists(dst):
            shutil.move(src, dst)
            moved += 1
        else:
            # Se já existe, remove da origem para manter consistência
            if os.path.exists(src):
                os.remove(src)
            skipped += 1
    
    return moved, skipped

In [ ]:
# Limpeza de dados: Remove arquivos problemáticos que podem quebrar o treinamento

# 1. Remove arquivos vazios (0 bytes)
# Imagens corrompidas frequentemente aparecem como arquivos de tamanho zero

!find /tmp/data/ -size 0 -exec rm {} +

# 2. Remove arquivos que não são JPG
# Garante que apenas imagens válidas sejam usadas no pipeline
# O dataset pode conter arquivos .db, .txt, etc. que causam erros no TensorFlow

!find /tmp/data/ -type f ! -name "*.jpg" -exec rm {} +

In [ ]:
# Relatório final: Contagem de imagens por classe e conjunto
# Importante para detectar desbalanceamento de classes

print("=" * 50)
print("CONJUNTO DE TREINO")
print("=" * 50)
print(f"Gatos:    {len(os.listdir(os.path.join(base_dir, 'train/cats')))} imagens")
print(f"Cães:     {len(os.listdir(os.path.join(base_dir, 'train/dogs')))} imagens")
print(f"Pássaros: {len(os.listdir(os.path.join(base_dir, 'train/birds')))} imagens")

print("\n" + "=" * 50)
print("CONJUNTO DE AVALIAÇÃO")
print("=" * 50)
print(f"Gatos:    {len(os.listdir(os.path.join(base_dir, 'eval/cats')))} imagens")
print(f"Cães:     {len(os.listdir(os.path.join(base_dir, 'eval/dogs')))} imagens")
print(f"Pássaros: {len(os.listdir(os.path.join(base_dir, 'eval/birds')))} imagens")

# Dica MLOps: Se houver grande desbalanceamento (ex: 10x mais cães que gatos),
# considere técnicas como class_weight, oversampling ou data augmentation

In [ ]:
"""
SIMULAÇÃO DE CENÁRIO REAL: Dataset Desbalanceado

Contexto MLOps:
---------------
Em produção, é comum ter datasets desbalanceados onde algumas classes têm muito
menos exemplos que outras. Exemplos reais:
- Detecção de fraudes (99% transações normais, 1% fraudes)
- Diagnóstico médico (doenças raras vs. casos comuns)
- Controle de qualidade (produtos defeituosos são minoria)

Neste experimento:
------------------
Simulamos um cenário onde parte das imagens de cães e pássaros foi "perdida"
(deletada, corrompida, ou não coletada):
- Gatos: 100% dos dados (classe majoritária)
- Cães: apenas 20% dos dados originais
- Pássaros: apenas 10% dos dados originais

Objetivo:
---------
Comparar o desempenho de modelos treinados em datasets balanceados vs. desbalanceados
e avaliar técnicas de mitigação (class weights, oversampling, data augmentation).
"""

# Criação da estrutura de diretórios para o dataset desbalanceado
imbalanced_dirs = [f'imbalanced/{dir_path}' for dir_path in train_eval_dirs]

for dir_path in imbalanced_dirs:
    full_path = os.path.join(base_dir, dir_path)
    if not os.path.exists(full_path):
        os.makedirs(full_path)

print("Estrutura de diretórios 'imbalanced' criada.\n")


# Função auxiliar para copiar (não mover) uma porcentagem das imagens
def copy_with_limit(origin, destination, percentage_split):
    """
    Copia uma porcentagem das imagens de origem para destino.
    
    Diferença da função anterior (move_to_destination):
    ----------------------------------------------------
    - Usa shutil.COPY em vez de shutil.MOVE
    - Preserva os dados originais intactos
    - Permite criar múltiplas versões do dataset (balanced, imbalanced, augmented)
    
    Parâmetros:
    -----------
    origin : str
        Diretório de origem
    destination : str
        Diretório de destino
    percentage_split : float
        Porcentagem de imagens a copiar (0.1 = 10%, 1.0 = 100%)
    """
    num_images = int(len(os.listdir(origin)) * percentage_split)
    
    for image_name, image_number in zip(sorted(os.listdir(origin)), range(num_images)):
        shutil.copy(os.path.join(origin, image_name), destination)


# Criação do dataset desbalanceado - TREINO
print("Criando conjunto de TREINO desbalanceado...")
print("  - Gatos: 100% (baseline)")
print("  - Cães: 20% (classe minoritária)")
print("  - Pássaros: 10% (classe muito minoritária)\n")

copy_with_limit(
    os.path.join(base_dir, 'train/cats'), 
    os.path.join(base_dir, 'imbalanced/train/cats'), 
    1.0  # 100% dos gatos
)
copy_with_limit(
    os.path.join(base_dir, 'train/dogs'), 
    os.path.join(base_dir, 'imbalanced/train/dogs'), 
    0.2  # 20% dos cães
)
copy_with_limit(
    os.path.join(base_dir, 'train/birds'), 
    os.path.join(base_dir, 'imbalanced/train/birds'), 
    0.1  # 10% dos pássaros
)

# Criação do dataset desbalanceado - AVALIAÇÃO
print("Criando conjunto de AVALIAÇÃO desbalanceado (mesmas proporções)...\n")

copy_with_limit(
    os.path.join(base_dir, 'eval/cats'), 
    os.path.join(base_dir, 'imbalanced/eval/cats'), 
    1.0
)
copy_with_limit(
    os.path.join(base_dir, 'eval/dogs'), 
    os.path.join(base_dir, 'imbalanced/eval/dogs'), 
    0.2
)
copy_with_limit(
    os.path.join(base_dir, 'eval/birds'), 
    os.path.join(base_dir, 'imbalanced/eval/birds'), 
    0.1
)

In [ ]:
# Relatório de distribuição do dataset desbalanceado
# Importante para calcular class_weights e avaliar estratégias de balanceamento

print("=" * 60)
print("DATASET DESBALANCEADO - CONJUNTO DE TREINO")
print("=" * 60)

train_cats_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/train/cats')))
train_dogs_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/train/dogs')))
train_birds_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/train/birds')))

print(f"Gatos:    {train_cats_imb:>6} imagens (100%)")
print(f"Cães:     {train_dogs_imb:>6} imagens ( 20%)")
print(f"Pássaros: {train_birds_imb:>6} imagens ( 10%)")
print(f"{'─' * 60}")
print(f"Total:    {train_cats_imb + train_dogs_imb + train_birds_imb:>6} imagens")

# Cálculo da razão de desbalanceamento (imbalance ratio)
# Métrica importante para decidir estratégia de mitigação
max_class = train_cats_imb
imbalance_ratio_dogs = max_class / train_dogs_imb
imbalance_ratio_birds = max_class / train_birds_imb

print(f"\nRazão de desbalanceamento:")
print(f"  Gatos/Cães:     {imbalance_ratio_dogs:.1f}:1")
print(f"  Gatos/Pássaros: {imbalance_ratio_birds:.1f}:1")

print("\n" + "=" * 60)
print("DATASET DESBALANCEADO - CONJUNTO DE AVALIAÇÃO")
print("=" * 60)

eval_cats_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/eval/cats')))
eval_dogs_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/eval/dogs')))
eval_birds_imb = len(os.listdir(os.path.join(base_dir, 'imbalanced/eval/birds')))

print(f"Gatos:    {eval_cats_imb:>6} imagens (100%)")
print(f"Cães:     {eval_dogs_imb:>6} imagens ( 20%)")
print(f"Pássaros: {eval_birds_imb:>6} imagens ( 10%)")
print(f"{'─' * 60}")
print(f"Total:    {eval_cats_imb + eval_dogs_imb + eval_birds_imb:>6} imagens")

In [ ]:
from tensorflow.keras import layers, models, optimizers

def create_model():
    """
    Cria uma CNN (Convolutional Neural Network) para classificação de imagens.
    
    Arquitetura:
    ------------
    Modelo sequencial com 4 blocos convolucionais + classificador denso
    
    Blocos Convolucionais:
    - Bloco 1: 32 filtros 3x3 → MaxPooling 2x2 (extrai features básicas: bordas, texturas)
    - Bloco 2: 64 filtros 3x3 → MaxPooling 2x2 (features intermediárias: formas)
    - Bloco 3: 64 filtros 3x3 → MaxPooling 2x2 (padrões mais complexos)
    - Bloco 4: 128 filtros 3x3 → MaxPooling 2x2 (features de alto nível)
    
    Classificador:
    - Flatten: Converte feature maps 3D em vetor 1D
    - Dense(512): Camada totalmente conectada com 512 neurônios
    - Dense(3): Camada de saída com 3 classes (cats, dogs, birds)
    
    Parâmetros:
    -----------
    - Input shape: (150, 150, 3) - imagens RGB de 150x150 pixels
    - Activation: ReLU nas camadas ocultas, Softmax na saída
    - Loss: SparseCategoricalCrossentropy (para labels inteiros: 0, 1, 2)
    - Optimizer: Adam (learning rate padrão = 0.001)
    - Metrics: SparseCategoricalAccuracy (acurácia de classificação)
    
    Returns:
    --------
    model : tf.keras.Model
        Modelo compilado e pronto para treinamento
    """
    
    # Definição da arquitetura sequencial
    model = models.Sequential([
        # Bloco Convolucional 1: Extração de features básicas
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2,2)),
        
        # Bloco Convolucional 2: Features intermediárias
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        
        # Bloco Convolucional 3: Padrões mais complexos
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        
        # Bloco Convolucional 4: Features de alto nível
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        
        # Classificador
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    # Compilação do modelo
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer=optimizers.Adam(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

In [ ]:
# Instanciação do modelo para o experimento com dataset desbalanceado
imbalanced_model = create_model()

# Exibe a arquitetura e contagem de parâmetros
print("=" * 70)
print("ARQUITETURA DO MODELO - EXPERIMENTO IMBALANCED")
print("=" * 70)
print(imbalanced_model.summary())
print("\n⚠️  Nota: Este modelo será treinado com o dataset DESBALANCEADO")
print("   (sem class weights ou técnicas de balanceamento)")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

"""
Data Generators: Pipeline de carregamento e pré-processamento de imagens

Vantagens do ImageDataGenerator:
---------------------------------
1. Carregamento sob demanda (não precisa carregar tudo na memória)
2. Normalização automática dos pixels
3. Suporte a data augmentation (rotação, zoom, flip, etc.)
4. Integração nativa com model.fit()

Configuração atual:
-------------------
- rescale=1./255: Normaliza pixels de [0, 255] para [0, 1]
  (Redes neurais convergem melhor com valores pequenos)
- SEM data augmentation (será adicionado em experimentos futuros)
"""

# Generator para TREINO: apenas normalização
train_datagen = ImageDataGenerator(rescale=1./255)

# Generator para VALIDAÇÃO: apenas normalização (nunca aplicar augmentation em eval!)
test_datagen = ImageDataGenerator(rescale=1./255)

# Configuração do fluxo de dados de TREINO
train_generator = train_datagen.flow_from_directory(
    '/tmp/data/imbalanced/train',  # Diretório raiz (contém subpastas cats, dogs, birds)
    target_size=(150, 150),         # Redimensiona todas as imagens para 150x150
    batch_size=32,                  # Número de imagens por batch (ajustar conforme memória)
    class_mode='sparse'             # Labels como inteiros (0, 1, 2) em vez de one-hot
)

# Configuração do fluxo de dados de VALIDAÇÃO
validation_generator = test_datagen.flow_from_directory(
    '/tmp/data/imbalanced/eval',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)


In [ ]:
# Validação do mapeamento de classes
print("=" * 70)
print("MAPEAMENTO DE CLASSES (Class Indices)")
print("=" * 70)
print(f"Treino:    {train_generator.class_indices}")
print(f"Validação: {validation_generator.class_indices}")
print("\nInterpretação:")
print("  - O número representa o label que o modelo vai prever")
print("  - Exemplo: se o modelo prever '1', significa 'dog'")
print("=" * 70)

# Informações adicionais sobre os generators
print(f"\nTotal de imagens de treino: {train_generator.samples}")
print(f"Total de imagens de validação: {validation_generator.samples}")
print(f"Número de batches por época (treino): {len(train_generator)}")
print(f"Número de batches por época (validação): {len(validation_generator)}")

In [ ]:
# Load pretrained model and history
imbalanced_history = pd.read_csv('/tmp/history-imbalanced/history-imbalanced.csv')
imbalanced_model = tf.keras.models.load_model('/tmp/model-imbalanced')  

print("✅ Modelo e histórico carregados com sucesso!")
print(f"Épocas treinadas: {len(imbalanced_history)}")

In [ ]:
def get_training_metrics(history):
    """
    Extrai métricas de treinamento de um objeto History ou DataFrame.
    
    Parâmetros:
    -----------
    history : keras.callbacks.History ou pd.DataFrame
        Histórico de treinamento do modelo
    
    Retorna:
    --------
    acc, val_acc, loss, val_loss : listas ou pd.Series
        Métricas de acurácia e loss para treino e validação
    """
    
    if not isinstance(history, pd.core.frame.DataFrame):
        history = history.history
    
    acc = history['sparse_categorical_accuracy']
    val_acc = history['val_sparse_categorical_accuracy']
    
    loss = history['loss']
    val_loss = history['val_loss']
    
    return acc, val_acc, loss, val_loss

In [ ]:
def plot_train_eval(history):
    """
    Plota curvas de acurácia e loss para treino vs. validação.
    
    Visualização essencial em MLOps para detectar:
    - Overfitting: treino continua melhorando, validação piora
    - Underfitting: ambos não melhoram
    - Convergência: ambos estabilizam em valores próximos
    
    Parâmetros:
    -----------
    history : keras.callbacks.History ou pd.DataFrame
        Histórico de treinamento do modelo
    """
    acc, val_acc, loss, val_loss = get_training_metrics(history)
    
    # Gráfico de Acurácia

    acc_plot_data = pd.DataFrame({
        "training accuracy": acc, 
        "evaluation accuracy": val_acc
    })
    
    plt.figure(figsize=(12, 5))  # Cria figura com 2 subplots lado a lado
    
    # Subplot 1: Acurácia
    plt.subplot(1, 2, 1)
    sns.lineplot(data=acc_plot_data)
    plt.title('Training vs Evaluation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Sparse Categorical Accuracy')
    plt.legend(['Training', 'Evaluation'])
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Loss
    plt.subplot(1, 2, 2)

    loss_plot_data = pd.DataFrame({
        "training loss": loss, 
        "evaluation loss": val_loss
    })
    sns.lineplot(data=loss_plot_data)
    plt.title('Training vs Evaluation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Evaluation'])
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Chama a função para plotar
plot_train_eval(imbalanced_history)

In [ ]:
from sklearn.metrics import (
    confusion_matrix, 
    ConfusionMatrixDisplay, 
    accuracy_score, 
    balanced_accuracy_score,
    classification_report
)

"""
AVALIAÇÃO DO MODELO IMBALANCED

Objetivo:
---------
Avaliar o desempenho do modelo treinado com dataset desbalanceado usando:
1. Accuracy Score (métrica padrão, mas enganosa em datasets desbalanceados)
2. Balanced Accuracy Score (compensa o desbalanceamento)
3. Confusion Matrix (visualiza erros por classe)
4. Proporção de erros por classe (identifica classes problemáticas)

Por que shuffle=False?
----------------------
Para calcular métricas, precisamos que a ordem das predições corresponda
exatamente à ordem dos labels verdadeiros. Com shuffle=True, essa correspondência
seria perdida.
"""

# Cria generator SEM shuffle para manter correspondência entre predições e labels
val_gen_no_shuffle = test_datagen.flow_from_directory(
    '/tmp/data/imbalanced/eval',  # Dataset desbalanceado
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False  # CRÍTICO: mantém ordem para cálculo de métricas
)

# Extrai os labels verdadeiros do generator
y_true = val_gen_no_shuffle.classes

print("Gerando predições no conjunto de validação...")
print("(Isso pode levar alguns minutos dependendo do tamanho do dataset)\n")

# Gera predições para todo o conjunto de validação
predictions_imbalanced = imbalanced_model.predict(val_gen_no_shuffle)

# Converte probabilidades (softmax) em classes preditas
# predictions_imbalanced shape: (n_samples, 3) → valores entre 0 e 1
# y_pred_imbalanced shape: (n_samples,) → valores 0, 1, ou 2
y_pred_imbalanced = np.argmax(predictions_imbalanced, axis=1)

print("=" * 70)
print("MÉTRICAS DE AVALIAÇÃO - MODELO IMBALANCED")
print("=" * 70)

# Accuracy Score: % de predições corretas (enganosa em datasets desbalanceados)
acc_score = accuracy_score(y_true, y_pred_imbalanced)
print(f"Accuracy Score:          {acc_score:.4f} ({acc_score*100:.2f}%)")

# Balanced Accuracy Score: média das acurácias por classe (melhor para imbalance)
balanced_acc = balanced_accuracy_score(y_true, y_pred_imbalanced)
print(f"Balanced Accuracy Score: {balanced_acc:.4f} ({balanced_acc*100:.2f}%)")

print("\n⚠️  Nota: Em datasets desbalanceados, Balanced Accuracy é mais confiável")
print("   que Accuracy padrão, pois dá peso igual a todas as classes.\n")

In [ ]:
# Calcula e plota a Confusion Matrix
imbalanced_cm = confusion_matrix(y_true, y_pred_imbalanced)

plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay(
    imbalanced_cm, 
    display_labels=['birds', 'cats', 'dogs']
).plot(values_format="d", cmap='Blues')
plt.title('Confusion Matrix - Modelo Imbalanced\n(Dataset de Validação Desbalanceado)')
plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("INTERPRETAÇÃO DA CONFUSION MATRIX")
print("=" * 70)
print("Linhas: Classes verdadeiras (ground truth)")
print("Colunas: Classes preditas pelo modelo")
print("Diagonal: Predições corretas")
print("Fora da diagonal: Erros de classificação\n")

# Calcula proporção de erros por classe
# Fórmula: (soma dos erros da linha) / (total de exemplos da classe)
misclassified_birds = (imbalanced_cm[0, 1] + imbalanced_cm[0, 2]) / np.sum(imbalanced_cm, axis=1)[0]
misclassified_cats = (imbalanced_cm[1, 0] + imbalanced_cm[1, 2]) / np.sum(imbalanced_cm, axis=1)[1]
misclassified_dogs = (imbalanced_cm[2, 0] + imbalanced_cm[2, 1]) / np.sum(imbalanced_cm, axis=1)[2]

print("=" * 70)
print("PROPORÇÃO DE ERROS POR CLASSE")
print("=" * 70)
print(f"Pássaros (birds): {misclassified_birds*100:.2f}% de erros")
print(f"Gatos (cats):     {misclassified_cats*100:.2f}% de erros")
print(f"Cães (dogs):      {misclassified_dogs*100:.2f}% de erros")

# Identifica a classe com maior taxa de erro
classes = ['birds', 'cats', 'dogs']
errors = [misclassified_birds, misclassified_cats, misclassified_dogs]
worst_class = classes[np.argmax(errors)]
worst_error = max(errors) * 100

print(f"\n⚠️  Classe mais problemática: {worst_class} ({worst_error:.2f}% de erros)")
print("   Isso é esperado em datasets desbalanceados: classes minoritárias")
print("   têm menos exemplos para aprender, resultando em mais erros.\n")

# Classification Report (métricas detalhadas por classe)
print("=" * 70)
print("CLASSIFICATION REPORT (Precision, Recall, F1-Score por classe)")
print("=" * 70)
print(classification_report(
    y_true, 
    y_pred_imbalanced, 
    target_names=['birds', 'cats', 'dogs'],
    digits=4
))

In [ ]:
"""
EXPERIMENTO DE CONTROLE: Modelo Baseline Ingênuo

Contexto MLOps:
---------------
Para demonstrar o problema do desbalanceamento, criamos um "modelo" que sempre
prediz a classe majoritária (cats). Isso mostra como a métrica Accuracy pode
ser enganosa em datasets desbalanceados.

Resultado esperado:
-------------------
- Accuracy alta (porque a maioria das imagens são gatos)
- Balanced Accuracy baixa (porque erra 100% das outras classes)

Lição:
------
Sempre use múltiplas métricas em MLOps. Accuracy sozinha pode esconder problemas graves.
"""

print("=" * 70)
print("EXPERIMENTO: Modelo que sempre prediz 'CATS'")
print("=" * 70)
print("Objetivo: Demonstrar por que Accuracy sozinha é enganosa\n")

# Cria array de predições: tudo = 1 (cats)
all_cats = np.ones(y_true.shape)

# Calcula métricas
acc_all_cats = accuracy_score(y_true, all_cats)
balanced_acc_all_cats = balanced_accuracy_score(y_true, all_cats)

print(f"Accuracy Score:          {acc_all_cats:.4f} ({acc_all_cats*100:.2f}%)")
print(f"Balanced Accuracy Score: {balanced_acc_all_cats:.4f} ({balanced_acc_all_cats*100:.2f}%)")

print("\n" + "⚠️ " * 35)
print("ANÁLISE CRÍTICA:")
print(f"  - Um modelo 'burro' que sempre chuta 'cats' tem {acc_all_cats*100:.2f}% de accuracy!")
print(f"  - Mas Balanced Accuracy revela a verdade: apenas {balanced_acc_all_cats*100:.2f}%")
print("  - Isso acontece porque o dataset tem muito mais gatos que outras classes")
print("\nCONCLUSÃO MLOps:")
print("  ✅ Sempre use Balanced Accuracy, F1-Score, ou Recall por classe")
print("  ❌ NUNCA confie apenas em Accuracy para datasets desbalanceados")
print("⚠️ " * 35 + "\n")

In [ ]:
"""
COMPARAÇÃO: Avaliação do mesmo modelo em dataset BALANCEADO

Objetivo:
---------
Avaliar como o modelo imbalanced se comporta quando testado em um dataset
balanceado (distribuição igual de cats, dogs, birds).

Hipótese:
---------
O modelo deve ter desempenho pior em classes minoritárias (dogs, birds),
pois foi treinado com poucos exemplos dessas classes.
"""

print("=" * 70)
print("AVALIAÇÃO EM DATASET BALANCEADO (para comparação)")
print("=" * 70)

# Cria generator para o dataset balanceado de validação
val_gen_balanced = test_datagen.flow_from_directory(
    '/tmp/data/eval',  # Dataset balanceado original
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

# Extrai labels verdadeiros
y_true_balanced = val_gen_balanced.classes

print("Gerando predições no conjunto de validação BALANCEADO...\n")

# Gera predições
predictions_balanced = imbalanced_model.predict(val_gen_balanced)
y_pred_balanced = np.argmax(predictions_balanced, axis=1)

# Calcula métricas
acc_balanced = accuracy_score(y_true_balanced, y_pred_balanced)
balanced_acc_balanced = balanced_accuracy_score(y_true_balanced, y_pred_balanced)

print(f"Accuracy Score:          {acc_balanced:.4f} ({acc_balanced*100:.2f}%)")
print(f"Balanced Accuracy Score: {balanced_acc_balanced:.4f} ({balanced_acc_balanced*100:.2f}%)")

# Confusion Matrix
cm_balanced = confusion_matrix(y_true_balanced, y_pred_balanced)

plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay(
    cm_balanced, 
    display_labels=['birds', 'cats', 'dogs']
).plot(values_format="d", cmap='Greens')
plt.title('Confusion Matrix - Modelo Imbalanced\n(Dataset de Validação BALANCEADO)')
plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("COMPARAÇÃO: Imbalanced vs Balanced Evaluation Set")
print("=" * 70)
print(f"Balanced Accuracy (eval imbalanced): {balanced_acc:.4f}")
print(f"Balanced Accuracy (eval balanced):   {balanced_acc_balanced:.4f}")
print(f"Diferença: {abs(balanced_acc - balanced_acc_balanced):.4f}")
print("\n💡 Insight: A diferença mostra o impacto do desbalanceamento no desempenho real.")
print("=" * 70)

In [ ]:
"""
EXPERIMENTO 2: MODELO TREINADO COM DATASET BALANCEADO

Objetivo:
---------
Treinar o mesmo modelo (mesma arquitetura) com um dataset balanceado
(distribuição igual de cats, dogs, birds) e comparar com o modelo imbalanced.

Hipótese:
---------
O modelo balanceado deve ter:
- Balanced Accuracy maior
- Desempenho mais uniforme entre as classes
- Menos viés para a classe majoritária

Configuração:
-------------
- Arquitetura: Idêntica ao modelo imbalanced (para comparação justa)
- Dataset: /tmp/data/train e /tmp/data/eval (balanceados)
- Sem data augmentation (será adicionado no Experimento 3)
"""

# Instancia um novo modelo com a mesma arquitetura
balanced_model = create_model()

# Configuração dos data generators (apenas normalização, sem augmentation)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generator de treino - dataset BALANCEADO
train_generator = train_datagen.flow_from_directory(
    '/tmp/data/train',  # Dataset balanceado
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

# Generator de validação - dataset BALANCEADO
validation_generator = test_datagen.flow_from_directory(
    '/tmp/data/eval',  # Dataset balanceado
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

print("=" * 70)
print("CONFIGURAÇÃO DO EXPERIMENTO - MODELO BALANCEADO")
print("=" * 70)
print(f"Treino - Total de imagens: {train_generator.samples}")
print(f"Treino - Distribuição: {train_generator.class_indices}")
print(f"\nValidação - Total de imagens: {validation_generator.samples}")
print(f"Validação - Distribuição: {validation_generator.class_indices}")
print("=" * 70)

In [ ]:
"""
CARREGAMENTO DO MODELO PRÉ-TREINADO BALANCEADO

Nota:
-----
Em vez de treinar do zero (o que levaria ~30-60 minutos), carregamos
um modelo já treinado com o dataset balanceado.

Em produção MLOps, você faria:
- Treinar com model.fit()
- Salvar com model.save()
- Versionar com MLflow ou DVC
- Registrar métricas e hiperparâmetros
"""

# Carrega o histórico de treinamento (CSV com métricas por época)
balanced_history = pd.read_csv('/tmp/history-balanced/history-balanced.csv')

# Carrega o modelo treinado (SavedModel format)
balanced_model = tf.keras.models.load_model('/tmp/model-balanced')

print("✅ Modelo balanceado e histórico carregados com sucesso!")
print(f"Épocas treinadas: {len(balanced_history)}")
print(f"Melhor val_accuracy: {balanced_history['val_sparse_categorical_accuracy'].max():.4f}")
print(f"Melhor val_loss: {balanced_history['val_loss'].min():.4f}")

In [ ]:
"""
AVALIAÇÃO DO MODELO BALANCEADO

Processo:
---------
1. Cria generator sem shuffle (para manter correspondência labels/predições)
2. Gera predições para todo o conjunto de validação
3. Calcula métricas (Accuracy, Balanced Accuracy)
4. Plota Confusion Matrix
"""

# Cria generator SEM shuffle para avaliação
val_gen_no_shuffle = test_datagen.flow_from_directory(
    '/tmp/data/eval',  # Dataset balanceado
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False  # CRÍTICO: mantém ordem para métricas
)

# Extrai labels verdadeiros
y_true = val_gen_no_shuffle.classes

print("Gerando predições no conjunto de validação balanceado...")
print("(Isso pode levar alguns minutos)\n")

# Gera predições
predictions_balanced = balanced_model.predict(val_gen_no_shuffle)

# Converte probabilidades (softmax) em classes preditas
y_pred_balanced = np.argmax(predictions_balanced, axis=1)

print("=" * 70)
print("MÉTRICAS DE AVALIAÇÃO - MODELO BALANCEADO")
print("=" * 70)

# Accuracy Score
acc_balanced = accuracy_score(y_true, y_pred_balanced)
print(f"Accuracy Score:          {acc_balanced:.4f} ({acc_balanced*100:.2f}%)")

# Balanced Accuracy Score
balanced_acc_balanced = balanced_accuracy_score(y_true, y_pred_balanced)
print(f"Balanced Accuracy Score: {balanced_acc_balanced:.4f} ({balanced_acc_balanced*100:.2f}%)")

print("\n💡 Nota: Com dataset balanceado, Accuracy e Balanced Accuracy")
print("   devem ser próximas (diferença < 2-3%).\n")

# Confusion Matrix
balanced_cm = confusion_matrix(y_true, y_pred_balanced)

plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay(
    balanced_cm, 
    display_labels=['birds', 'cats', 'dogs']
).plot(values_format="d", cmap='Greens')
plt.title('Confusion Matrix - Modelo Balanceado\n(Dataset de Validação Balanceado)')
plt.tight_layout()
plt.show()

# Análise de erros por classe
misclassified_birds = (balanced_cm[0, 1] + balanced_cm[0, 2]) / np.sum(balanced_cm, axis=1)[0]
misclassified_cats = (balanced_cm[1, 0] + balanced_cm[1, 2]) / np.sum(balanced_cm, axis=1)[1]
misclassified_dogs = (balanced_cm[2, 0] + balanced_cm[2, 1]) / np.sum(balanced_cm, axis=1)[2]

print("\n" + "=" * 70)
print("PROPORÇÃO DE ERROS POR CLASSE")
print("=" * 70)
print(f"Pássaros (birds): {misclassified_birds*100:.2f}% de erros")
print(f"Gatos (cats):     {misclassified_cats*100:.2f}% de erros")
print(f"Cães (dogs):      {misclassified_dogs*100:.2f}% de erros")

# Calcula desvio padrão dos erros (quanto mais baixo, mais uniforme)
errors = [misclassified_birds, misclassified_cats, misclassified_dogs]
error_std = np.std(errors)

print(f"\nDesvio padrão dos erros: {error_std:.4f}")
print("💡 Quanto menor o desvio, mais uniforme o desempenho entre classes.")

# Classification Report
print("\n" + "=" * 70)
print("CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(
    y_true,
    y_pred_balanced,
    target_names=['birds', 'cats', 'dogs'],
    digits=4
))

In [ ]:
augmented_history = pd.read_csv('/tmp/history-augmented/history-augmented.csv')
augmented_model = tf.keras.models.load_model('/tmp/model-augmented')

In [ ]:
"""
EXPERIMENTO 3: TREINAMENTO COM DATA AUGMENTATION

Objetivo:
---------
Combater overfitting através de data augmentation (aumento artificial do dataset).

O que é Data Augmentation?
---------------------------
Técnica que aplica transformações aleatórias nas imagens de treino para criar
variações artificiais, forçando o modelo a aprender features mais robustas.

Transformações aplicadas:
--------------------------
- rotation_range=50: Rotação aleatória de até 50 graus
- width_shift_range=0.15: Deslocamento horizontal de até 15%
- height_shift_range=0.15: Deslocamento vertical de até 15%
- shear_range=0.2: Distorção de cisalhamento de até 20%
- zoom_range=0.2: Zoom in/out de até 20%
- horizontal_flip=True: Espelhamento horizontal aleatório

Por que isso funciona?
-----------------------
- Aumenta artificialmente o tamanho do dataset
- Força o modelo a aprender features invariantes a transformações
- Reduz overfitting (modelo não "decora" as imagens exatas)
- Melhora generalização para imagens em condições variadas

IMPORTANTE:
-----------
Data augmentation é aplicado APENAS no treino, NUNCA na validação/teste!
"""

# Instancia um novo modelo com a mesma arquitetura
augmented_model = create_model()

# Generator de TREINO com data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalização (sempre necessária)
    rotation_range=50,           # Rotação: simula diferentes ângulos de captura
    width_shift_range=0.15,      # Shift horizontal: simula descentramento
    height_shift_range=0.15,     # Shift vertical: simula descentramento
    shear_range=0.2,             # Cisalhamento: simula perspectivas diferentes
    zoom_range=0.2,              # Zoom: simula diferentes distâncias
    horizontal_flip=True         # Espelhamento: dobra o dataset (gato da esquerda = gato da direita)
)

# Generator de VALIDAÇÃO sem augmentation (apenas normalização)
test_datagen = ImageDataGenerator(rescale=1./255)

# Cria os generators
train_generator = train_datagen.flow_from_directory(
    '/tmp/data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

validation_generator = test_datagen.flow_from_directory(
    '/tmp/data/eval',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

print("=" * 70)
print("CONFIGURAÇÃO DO EXPERIMENTO - MODELO COM DATA AUGMENTATION")
print("=" * 70)
print("Transformações aplicadas no TREINO:")
print("  - Rotação: até 50°")
print("  - Deslocamento: até 15% (horizontal e vertical)")
print("  - Cisalhamento: até 20%")
print("  - Zoom: até 20%")
print("  - Espelhamento horizontal: Sim")
print("\nValidação: APENAS normalização (sem augmentation)")
print("=" * 70)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import random

def display_transformations(gen, title="Transformações Aplicadas"):
    """
    Visualiza exemplos de transformações aplicadas por um ImageDataGenerator.
    
    Parâmetros:
    -----------
    gen : ImageDataGenerator
        Generator configurado com transformações
    title : str
        Título para o conjunto de transformações
    
    Utilidade MLOps:
    ----------------
    Permite validar visualmente se as transformações são realistas.
    Transformações muito extremas podem gerar imagens irreconhecíveis,
    prejudicando o treinamento em vez de ajudar.
    """
    train_birds_dir = "/tmp/data/train/birds"
    
    # Seleciona uma imagem aleatória de pássaro
    bird_images = os.listdir(train_birds_dir)
    random_index = random.randint(0, len(bird_images) - 1)
    sample_image_path = os.path.join(train_birds_dir, bird_images[random_index])
    
    # Carrega e prepara a imagem
    sample_image = load_img(sample_image_path, target_size=(150, 150))
    sample_array = img_to_array(sample_image)
    sample_array = sample_array[None, :]  # Adiciona dimensão de batch
    
    print("=" * 70)
    print(title)
    print("=" * 70)
    print(f"Imagem original: {bird_images[random_index]}\n")
    
    # Exibe a imagem original
    print("IMAGEM ORIGINAL:")
    display(sample_image)
    
    # Gera e exibe 4 transformações aleatórias
    print("\nTRANSFORMAÇÕES GERADAS:")
    for iteration, array in zip(range(4), gen.flow(sample_array, batch_size=1)):
        array = np.squeeze(array)  # Remove dimensão de batch
        img = array_to_img(array)
        print(f"\nTransformação #{iteration + 1}:")
        display(img)
    
    print("\n" + "=" * 70 + "\n")


# Visualiza transformações MODERADAS (usadas no treinamento)
print("🔍 ANÁLISE: Transformações Moderadas (Configuração de Treino)")
print("Objetivo: Verificar se as transformações mantêm as imagens reconhecíveis\n")

sample_gen_moderate = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True
)

display_transformations(sample_gen_moderate, "TRANSFORMAÇÕES MODERADAS")

print("✅ Análise: As imagens transformadas ainda são reconhecíveis?")
print("   - Se SIM: Configuração adequada para treinamento")
print("   - Se NÃO: Reduzir intensidade das transformações\n")

In [ ]:
"""
COMPARAÇÃO: Transformações EXTREMAS (não recomendadas)

Objetivo:
---------
Demonstrar o que acontece quando data augmentation é muito agressivo.

Problema:
---------
Transformações extremas podem:
- Gerar imagens irreconhecíveis (até para humanos)
- Confundir o modelo em vez de ajudar
- Introduzir artefatos que não existem em dados reais
- Prejudicar o desempenho em vez de melhorar

Regra de ouro:
--------------
Se um humano não consegue reconhecer a classe na imagem transformada,
o modelo também terá dificuldade.
"""

print("⚠️  ANÁLISE: Transformações Extremas (NÃO recomendadas)")
print("Objetivo: Demonstrar os limites do data augmentation\n")

# Generator com transformações MUITO agressivas
sample_gen_extreme = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,           # Rotação de até 90° (imagem de cabeça para baixo)
    width_shift_range=0.3,       # Deslocamento de 30% (pode cortar partes importantes)
    height_shift_range=0.3,      # Deslocamento de 30%
    shear_range=0.5,             # Cisalhamento extremo (distorce muito a forma)
    zoom_range=0.5,              # Zoom de 50% (pode perder contexto)
    vertical_flip=True,          # Espelhamento vertical (animais de cabeça para baixo)
    horizontal_flip=True
)

display_transformations(sample_gen_extreme, "TRANSFORMAÇÕES EXTREMAS (NÃO USAR)")

print("❌ Análise: As imagens transformadas estão irreconhecíveis?")
print("   - vertical_flip=True: Animais de cabeça para baixo (não realista)")
print("   - rotation_range=90: Pode gerar orientações impossíveis")
print("   - shear_range=0.5: Distorce demais a forma original")
print("\n💡 Lição MLOps:")
print("   Data augmentation deve simular VARIAÇÕES REALISTAS, não criar")
print("   imagens artificiais que nunca ocorreriam em produção.\n")

In [ ]:
"""
CARREGAMENTO DO MODELO PRÉ-TREINADO COM DATA AUGMENTATION

Nota:
-----
Carregamos um modelo já treinado com data augmentation para economizar tempo.

Expectativa:
------------
- Menor overfitting (gap menor entre train e eval)
- Curvas de treino mais "ruidosas" (devido às transformações aleatórias)
- Melhor generalização para imagens em condições variadas
"""

# Carrega o histórico de treinamento
augmented_history = pd.read_csv('/tmp/history-augmented/history-augmented.csv')

# Carrega o modelo treinado
augmented_model = tf.keras.models.load_model('/tmp/model-augmented')

print("✅ Modelo com data augmentation e histórico carregados com sucesso!")
print(f"Épocas treinadas: {len(augmented_history)}")
print(f"Melhor val_accuracy: {augmented_history['val_sparse_categorical_accuracy'].max():.4f}")
print(f"Melhor val_loss: {augmented_history['val_loss'].min():.4f}")

In [ ]:
"""
AVALIAÇÃO DO MODELO TREINADO COM DATA AUGMENTATION

Hipótese:
---------
O modelo com augmentation deve ter:
- Melhor generalização (menor gap entre train e eval)
- Desempenho similar ou melhor que o modelo balanceado
- Maior robustez a variações nas imagens de teste
"""

# Cria generator SEM shuffle para avaliação
val_gen_no_shuffle = test_datagen.flow_from_directory(
    '/tmp/data/eval',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

# Extrai labels verdadeiros
y_true = val_gen_no_shuffle.classes

print("Gerando predições no conjunto de validação...")
print("(Isso pode levar alguns minutos)\n")

# Gera predições
predictions_augmented = augmented_model.predict(val_gen_no_shuffle)
y_pred_augmented = np.argmax(predictions_augmented, axis=1)

print("=" * 70)
print("MÉTRICAS DE AVALIAÇÃO - MODELO COM DATA AUGMENTATION")
print("=" * 70)

# Accuracy Score
acc_augmented = accuracy_score(y_true, y_pred_augmented)
print(f"Accuracy Score:          {acc_augmented:.4f} ({acc_augmented*100:.2f}%)")

# Balanced Accuracy Score
balanced_acc_augmented = balanced_accuracy_score(y_true, y_pred_augmented)
print(f"Balanced Accuracy Score: {balanced_acc_augmented:.4f} ({balanced_acc_augmented*100:.2f}%)")

# Confusion Matrix
augmented_cm = confusion_matrix(y_true, y_pred_augmented)

plt.figure(figsize=(8, 6))
ConfusionMatrixDisplay(
    augmented_cm,
    display_labels=['birds', 'cats', 'dogs']
).plot(values_format="d", cmap='Purples')
plt.title('Confusion Matrix - Modelo com Data Augmentation\n(Dataset de Validação Balanceado)')
plt.tight_layout()
plt.show()

# Análise de erros por classe
misclassified_birds_aug = (augmented_cm[0, 1] + augmented_cm[0, 2]) / np.sum(augmented_cm, axis=1)[0]
misclassified_cats_aug = (augmented_cm[1, 0] + augmented_cm[1, 2]) / np.sum(augmented_cm, axis=1)[1]
misclassified_dogs_aug = (augmented_cm[2, 0] + augmented_cm[2, 1]) / np.sum(augmented_cm, axis=1)[2]

print("\n" + "=" * 70)
print("PROPORÇÃO DE ERROS POR CLASSE")
print("=" * 70)
print(f"Pássaros (birds): {misclassified_birds_aug*100:.2f}% de erros")
print(f"Gatos (cats):     {misclassified_cats_aug*100:.2f}% de erros")
print(f"Cães (dogs):      {misclassified_dogs_aug*100:.2f}% de erros")

# Classification Report
print("\n" + "=" * 70)
print("CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(
    y_true,
    y_pred_augmented,
    target_names=['birds', 'cats', 'dogs'],
    digits=4
))

In [ ]:
"""
VISUALIZAÇÃO DAS CURVAS DE TREINAMENTO - MODELO COM AUGMENTATION

Análise esperada:
-----------------
- Curvas de treino mais "ruidosas" (devido às transformações aleatórias)
- Gap menor entre train e eval (menos overfitting)
- Convergência pode ser mais lenta (modelo precisa aprender features mais robustas)
"""

print("=" * 70)
print("CURVAS DE TREINAMENTO - MODELO COM DATA AUGMENTATION")
print("=" * 70)

plot_train_eval(augmented_history)

print("\n💡 Observações esperadas:")
print("   - Curva de treino mais 'ruidosa' (variações entre épocas)")
print("   - Gap menor entre treino e validação (menos overfitting)")
print("   - Accuracy de treino pode ser MENOR que no modelo sem augmentation")
print("     (isso é NORMAL e DESEJÁVEL - significa que o modelo não está decorando)")

In [ ]:
"""
COMPARAÇÃO FINAL: IMBALANCED vs BALANCED vs AUGMENTED

Objetivo:
---------
Quantificar o impacto de cada técnica no desempenho do modelo.

Métricas comparadas:
--------------------
- Balanced Accuracy (métrica principal para datasets desbalanceados)
- Erro por classe (uniformidade do desempenho)
- Gap train-eval (indicador de overfitting)
"""

print("=" * 70)
print("COMPARAÇÃO FINAL: 3 EXPERIMENTOS")
print("=" * 70)

# Cria DataFrame para comparação
comparison_data = {
    'Modelo': ['Imbalanced', 'Balanced', 'Augmented'],
    'Balanced Accuracy': [
        balanced_acc,           # Do experimento 1
        balanced_acc_balanced,  # Do experimento 2
        balanced_acc_augmented  # Do experimento 3
    ],
    'Erro Birds (%)': [
        misclassified_birds * 100,
        (balanced_cm[0, 1] + balanced_cm[0, 2]) / np.sum(balanced_cm, axis=1)[0] * 100,
        misclassified_birds_aug * 100
    ],
    'Erro Cats (%)': [
        misclassified_cats * 100,
        (balanced_cm[1, 0] + balanced_cm[1, 2]) / np.sum(balanced_cm, axis=1)[1] * 100,
        misclassified_cats_aug * 100
    ],
    'Erro Dogs (%)': [
        misclassified_dogs * 100,
        (balanced_cm[2, 0] + balanced_cm[2, 1]) / np.sum(balanced_cm, axis=1)[2] * 100,
        misclassified_dogs_aug * 100
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Visualização gráfica
plt.figure(figsize=(14, 5))

# Subplot 1: Balanced Accuracy
plt.subplot(1, 2, 1)
plt.bar(comparison_df['Modelo'], comparison_df['Balanced Accuracy'], 
        color=['#ff6b6b', '#4ecdc4', '#95e1d3'])
plt.title('Balanced Accuracy - Comparação dos 3 Modelos')
plt.ylabel('Balanced Accuracy')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)

# Adiciona valores nas barras
for i, v in enumerate(comparison_df['Balanced Accuracy']):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# Subplot 2: Erro por classe
plt.subplot(1, 2, 2)
x = np.arange(len(comparison_df['Modelo']))
width = 0.25

plt.bar(x - width, comparison_df['Erro Birds (%)'], width, label='Birds', color='#ff6b6b')
plt.bar(x, comparison_df['Erro Cats (%)'], width, label='Cats', color='#4ecdc4')
plt.bar(x + width, comparison_df['Erro Dogs (%)'], width, label='Dogs', color='#95e1d3')

plt.title('Taxa de Erro por Classe - Comparação dos 3 Modelos')
plt.ylabel('Taxa de Erro (%)')
plt.xticks(x, comparison_df['Modelo'])
plt.legend()
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Análise final
print("\n" + "=" * 70)
print("CONCLUSÕES MLOps")
print("=" * 70)

best_model_idx = comparison_df['Balanced Accuracy'].idxmax()
best_model = comparison_df.loc[best_model_idx, 'Modelo']
best_score = comparison_df.loc[best_model_idx, 'Balanced Accuracy']

print(f"🏆 Melhor modelo: {best_model} (Balanced Accuracy: {best_score:.4f})")
print("\nLições aprendidas:")
print("  1. Desbalanceamento de classes prejudica MUITO o desempenho")
print("  2. Balancear o dataset é essencial para treinar modelos justos")
print("  3. Data augmentation ajuda a combater overfitting")
print("  4. Balanced Accuracy > Accuracy para datasets desbalanceados")
print("  5. Sempre analise métricas POR CLASSE, não apenas globais")
print("=" * 70)